# Movie Franchise Analysis
Questions to answer:
1. How many movies should there be per franchise?
2. When do you stop adding to a franchise?
3. Do non-numbered sequels perform better than numbered sequels?
4. How long to wait before you reboot?

In [1]:
from imdb import IMDb
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [2]:
ia = IMDb(accessSystem='http', adultSearch=False)

In [3]:
Fran_data=pd.read_csv('Franchise_dataset.csv')
Fran_data=Fran_data.drop('Unnamed: 0',axis=1)

In [4]:
html = 'http://www.boxofficemojo.com/franchises/chart/?id=zoolander.htm'
r = requests.get(html)
page_source=r.content
page_soup = bs(page_source, "lxml")

In [5]:
print("The number of tables in the document is ", len(page_soup.find_all('table')) )

The number of tables in the document is  7


In [6]:
# Table attributes: table border="0" cellspacing="1" cellpadding="5" width="95%"
no_cell_space = len(page_soup.find_all('table', attrs={'cellspacing': '1'}))
print("The number of tables in the document with this cellspacing is ", no_cell_space)

no_cell_pad = len(page_soup.find_all('table', attrs={'cellpadding': '5'}))
print( "The number of tables in the document with this cellpadding is ", no_cell_pad)

no_border = len(page_soup.find_all('table', attrs={'border': '0'}))
print( "The number of tables in the document with this border is ", no_border)

The number of tables in the document with this cellspacing is  3
The number of tables in the document with this cellpadding is  3
The number of tables in the document with this border is  7


In [7]:
table = page_soup.find_all('table', attrs={'cellspacing': '1'})[2]  
#check header row
data_row = table.find_all('tr')[0]
data_cols = data_row.find_all('td') 
# List of columns in the first data row
data_cols = data_cols
for column in data_cols:
    print( column.text)

Rank
Title (click to view)
Studio
Worldwide
Domestic / %
Overseas / %
Year


In [8]:
table = page_soup.find_all('table', attrs={'cellspacing': '1'})[2]  
#check header row
data_row = table.find_all('tr')[1]
data_cols = data_row.find_all('td')  # List of columns in the first data row
for column in data_cols:
    print( column.text)  # Access the text in each column

1
Zoolander
Par.
$60.8
$45.2
74.3%
$15.6
25.7%
2001


In [9]:
# Set up the dictionary to save the data
output_columns = ['Rank', 'Movie','Studio', 'Revenue', 'Domestic Revenue', 'Domestic %',
                  'Oversees Revenue', 'Oversees %', 'Year']
output = dict((x, []) for x in output_columns)
#Ignore first row coz its the header row
all_rows = table.find_all('tr')[1:3]
for row in all_rows:
    row_cols = row.find_all('td')
    # Loop through the columns and output keys to populate dictionary
    for dict_key, col in zip(output_columns, row_cols):
        output[dict_key].append(col.text)
        
# Put output into a DataFrame and rearrange columns in desired order
output_pd = pd.DataFrame(output)
output_pd = output_pd[output_columns]

In [10]:
output_pd 

,Rank,Movie,Studio,Revenue,Domestic Revenue,Domestic %,Oversees Revenue,Oversees %,Year
0,1,Zoolander,Par.,$60.8,$45.2,74.3%,$15.6,25.7%,2001
1,2,Zoolander 2,Par.,$56.7,$28.8,50.9%,$27.9,49.1%,2016


In [11]:
output_columns = ['Rank', 'Movie','Studio', 'Gross', 'Theatre', 'Opening',
                  'Opening Theatre', 'Date']
base='http://www.boxofficemojo.com/franchises'
output = dict((x, []) for x in output_columns)

for ID, total_movies in zip(Fran_data['FranchiseID'],Fran_data['Total Movies']):
    html = base+ID
    r = requests.get(html)
    page_source=r.content
    page_soup = bs(page_source, "lxml")
    table = page_soup.find_all('table', attrs={'cellspacing': '1'})[0]  
    all_rows = table.find_all('tr')[1:1+total_movies]
    
    for row in all_rows:
        row_cols = row.find_all('td')
    # Loop through the columns and output keys to populate dictionary
        for dict_key, col in zip(output_columns, row_cols):
            output[dict_key].append(col.text)
        
output_pd = pd.DataFrame(output)
output_pd = output_pd[output_columns]   

In [12]:
movie_data=output_pd

In [13]:
fran_id=[]
for ID, total_movies in zip(Fran_data['Franchise'],Fran_data['Total Movies']):
    for i in range(total_movies):
        fran_id.append(ID)

movie_data['Franchise']=fran_id

In [14]:
dates=movie_data['Date']
movie_year=[]
for date in dates:
    date_obj=datetime.strptime(date, '%m/%d/%y')
    movie_year.append(date_obj.year)
    
movie_data['Year']=movie_year

In [15]:
scrub_list=['Gross', 'Theatre', 'Opening', 'Opening Theatre']
for elem in scrub_list:
    for i in range(movie_data.shape[0]):
        if movie_data.loc[i,elem] == '-' or movie_data.loc[i,elem] == 'n/a':
            movie_data.loc[i,elem] = 0
        else:
            movie_data.loc[i,elem] = movie_data.loc[i,elem].strip('$')
            movie_data.loc[i,elem] = movie_data.loc[i,elem].replace(',','')
            movie_data.loc[i,elem] = int(movie_data.loc[i,elem])
        

In [16]:
movie_data

,Rank,Movie,Studio,Gross,Theatre,Opening,Opening Theatre,Date,Franchise,Year
0,1,300,WB,210614939,3280,70885301,3103,3/9/07,300,2007
1,2,300: Rise of An Empire,WB,106580051,3490,45038460,3470,3/7/14,300,2014
2,1,Alice in Wonderland (2010),BV,334191110,3739,116101023,3728,3/5/10,Alice in Wonderland,2010
3,2,Alice Through the Looking Glass,BV,77041381,3763,26858726,3763,5/27/16,Alice in Wonderland,2016
4,1,Alvin and the Chipmunks: The Squeakquel,Fox,219614612,3747,48875415,3700,12/23/09,Alvin and the Chipmunks,2009
5,2,Alvin and the Chipmunks,Fox,217326974,3499,44307417,3475,12/14/07,Alvin and the Chipmunks,2007
6,3,Alvin and the Chipmunks: Chipwrecked,Fox,133110742,3734,23244744,3723,12/16/11,Alvin and the Chipmunks,2011
7,4,Alvin and the Chipmunks: The Road Chip,Fox,85886987,3705,14287159,3653,12/18/15,Alvin and the Chipmunks,2015
8,5,The Chipmunk Adventure,Gold.,6804312,916,2584720,916,5/22/87,Alvin and the Chipmunks,1987
9,1,American Pie 2,Uni.,145103595,3157,45117985,3063,8/10/01,American Pie,2001


In [17]:
jk=movie_data['Franchise']=='J.K. Rowling\'s Wizarding World'
me=movie_data['Franchise']=='Middle Earth'
drop_list=np.where(jk)
drop_list=np.append(drop_list,[135,332,358])
drop_list=np.append(drop_list,np.where(me))
drop_list=drop_list.flatten()
drop_list


array([166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 135, 358, 236,
       237, 238, 239, 240, 241], dtype=int64)

In [18]:
movie_data=movie_data.drop(drop_list)
#split array
sp_index=np.split(np.array(movie_data.index),16)


In [20]:
#Add ratings
imdb_rating=[]
meta_score=[]
imdb_year=[]
# split index 0

for i in sp_index[0]:
    print(i)
    mov=ia.search_movie(movie_data.loc[i,'Movie'])
    if mov == []:
        imdb_year.append(3000)
        imdb_rating.append(0)
        meta_score.append(0)
    else:
        mov=ia.search_movie(movie_data.loc[i,'Movie'])[0]
        movie=ia.get_movie(mov.movieID)
        imdb_year.append(movie['year'])
        if 'rating' not in movie.keys():
            movie['rating']=0
        imdb_rating.append(movie['rating'])
        if movie['rating'] == 0:
            meta_score.append(0)
        else:
            m=ia.get_movie_critic_reviews(movie.movieID)
            if 'metascore' not in m['data'].keys():
                m['data']['metascore']='0'
            meta_score.append(int(m['data']['metascore']))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [59]:
movie_data['Imdb Year']=imdb_year
movie_data['Imdb Rating']=imdb_rating
movie_data['Meta Score']=meta_score

In [76]:
bad_year=movie_data[movie_data['Imdb Year'] != movie_data['Year'] ]


In [77]:
bad_score=movie_data[ movie_data['Meta Score'] ==0]

In [108]:
drop_index1=np.array(bad_year.index,dtype=int)
drop_index2=np.array(bad_score.index, dtype=int)
drop_index=np.array([drop_index1,drop_index2])
drop_index=drop_index.flatten()
drop_index=np.hstack((drop_index[0],drop_index[1]))
drop_index=np.unique(drop_index)
drop_index

array([  0,   8,  27,  33,  34, 101, 115, 116, 127, 130, 140, 141, 189,
       190, 191, 202, 206, 235, 250, 309, 332, 339, 342, 344, 364, 367,
       368, 370])

In [109]:
movie_data.loc[drop_index]

,Rank,Movie,Studio,Gross,Theatre,Opening,Opening Theatre,Date,Franchise,Year,Imdb Year,Imdb Rating,Meta Score
0,1,300,WB,210614939,3280,70885301,3103,3/9/07,300,2007.0,2006.0,7.7,52.0
8,5,The Chipmunk Adventure,Gold.,6804312,916,2584720,916,5/22/87,Alvin and the Chipmunks,1987.0,1987.0,7.2,0.0
27,4,Batman,WB,251188924,2201,40489746,2194,6/23/89,Batman,1989.0,1966.0,7.6,69.0
33,10,Batman: Mask of the Phantasm,WB,5617391,1506,1189975,1506,12/25/93,Batman,1993.0,1993.0,7.9,0.0
34,11,Batman: The Killing Joke,Fathom,3775000,1325,0,0,7/25/16,Batman,2016.0,2016.0,6.5,0.0
101,5,Fast and Furious,Uni.,155064265,3674,70950500,3461,4/3/09,The Fast and the Furious,2009.0,2001.0,6.6,46.0
115,1,Ghostbusters,Col.,229242989,1506,13578151,1339,6/8/84,Ghostbusters,1984.0,2016.0,7.8,71.0
116,2,Ghostbusters (2016),Sony,128350574,3963,46018755,3963,7/15/16,Ghostbusters,2016.0,2016.0,3.0,0.0
127,1,Harry Potter and the Deathly Hallows Part 2,WB,381011219,4375,169189427,4375,7/15/11,Harry Potter,2011.0,2016.0,8.1,87.0
130,4,Harry Potter and the Deathly Hallows Part 1,WB,295983305,4125,125017372,4125,11/19/10,Harry Potter,2010.0,2016.0,7.7,65.0


In [110]:
# Hard coded corrections
# Bad data index: [  0,   8,  27,  33,  34, 101, 115, 116, 127, 130, 140, 141, 189,190, 
#                  191, 202, 206, 235, 250, 309, 332, 339, 342, 344, 364, 367,368, 370])
# Index = 0 Wrong year no action required

# Index = 8 No metacritic score

# Index = 27 
movie_data.loc[27,'Imdb Rating']=7.6
movie_data.loc[27,'Meta Score']=69

# Index = 27 
movie_data.loc[27,'Imdb Rating']=7.6
movie_data.loc[27,'Meta Score']=69

# Index = 101
movie_data.loc[101,'Imdb Rating']=6.6
movie_data.loc[101,'Meta Score']=46

# Index = 115
movie_data.loc[115,'Imdb Rating']=7.8
movie_data.loc[115,'Meta Score']=71

# Index = 116
movie_data.loc[116,'Imdb Rating']=5.3
movie_data.loc[115,'Meta Score']=60

# Index = 127
movie_data.loc[127,'Imdb Rating']=8.1
movie_data.loc[127,'Meta Score']=87

# Index = 130
movie_data.loc[130,'Imdb Rating']=7.7
movie_data.loc[130,'Meta Score']=65

# Index = 140 No metacritic score
# Index = 141 No metacritic score
# Index = 189 No metacritic score
# Index = 190 No metacritic score
# Index = 191 wrong year
# Index = 206 No metacritic score

# Index = 202
movie_data.loc[202,'Imdb Rating']=7.6
movie_data.loc[202,'Meta Score']=67

# Index = 235
movie_data.loc[235,'Imdb Rating']=6.8
movie_data.loc[235,'Meta Score']=58

# Index = 250
movie_data.loc[250,'Imdb Rating']=7.0
movie_data.loc[250,'Meta Score']=48

# Index = 309
movie_data.loc[309,'Imdb Rating']=8.0
movie_data.loc[309,'Meta Score']=82

# Index = 332
#movie_data=movie_data.drop(332)

# Index = 339 wrong year

# Index = 342
movie_data.loc[342,'Imdb Rating']=7.8
movie_data.loc[342,'Meta Score']=50

# Index = 344 wrong year
# Index = 364 No metacritic score

# Index = 367
movie_data.loc[367,'Imdb Rating']=5.5
movie_data.loc[367,'Meta Score']=52

# Index = 368
movie_data.loc[368,'Imdb Rating']=4.9
movie_data.loc[368,'Meta Score']=45

# Index = 368 wrong year

In [111]:
movie_data

,Rank,Movie,Studio,Gross,Theatre,Opening,Opening Theatre,Date,Franchise,Year,Imdb Year,Imdb Rating,Meta Score
0,1,300,WB,210614939,3280,70885301,3103,3/9/07,300,2007,2006,7.7,52
1,2,300: Rise of An Empire,WB,106580051,3490,45038460,3470,3/7/14,300,2014,2014,6.2,48
2,1,Alice in Wonderland (2010),BV,334191110,3739,116101023,3728,3/5/10,Alice in Wonderland,2010,2010,6.5,53
3,2,Alice Through the Looking Glass,BV,77041381,3763,26858726,3763,5/27/16,Alice in Wonderland,2016,2016,6.2,34
4,1,Alvin and the Chipmunks: The Squeakquel,Fox,219614612,3747,48875415,3700,12/23/09,Alvin and the Chipmunks,2009,2009,4.5,41
5,2,Alvin and the Chipmunks,Fox,217326974,3499,44307417,3475,12/14/07,Alvin and the Chipmunks,2007,2007,5.2,39
6,3,Alvin and the Chipmunks: Chipwrecked,Fox,133110742,3734,23244744,3723,12/16/11,Alvin and the Chipmunks,2011,2011,4.4,24
7,4,Alvin and the Chipmunks: The Road Chip,Fox,85886987,3705,14287159,3653,12/18/15,Alvin and the Chipmunks,2015,2015,5.1,33
8,5,The Chipmunk Adventure,Gold.,6804312,916,2584720,916,5/22/87,Alvin and the Chipmunks,1987,1987,7.2,0
9,1,American Pie 2,Uni.,145103595,3157,45117985,3063,8/10/01,American Pie,2001,2001,6.4,43


In [112]:
np.where(movie_data['Imdb Rating']==0)[0]

array([], dtype=int64)

In [113]:
np.where(movie_data['Meta Score']==0)[0]

array([  8,  33,  34, 116, 139, 140, 178, 179, 195, 345], dtype=int64)

In [114]:
movie_data.to_csv('final_movie_dataset.csv')